# Import des librairies

In [2]:
import os
import io
import time
import datetime
import pandas as pd
import xlsxwriter
import numpy as np
import requests

# Init divers

In [2]:
list_df=list()

# Init du folder emplacement des fichiers CSV

In [13]:
filepath_data = r"C:\Users\gille\JupyterProjects\Foncier\data\external"+chr(92)
filepath_processed = r"C:\Users\gille\JupyterProjects\Foncier\data\processed"+chr(92)
filepath_interim = r"C:\Users\gille\JupyterProjects\Foncier\data\interim"+chr(92)
file_xls_analyse = "Analyse_dataframe_foncier.xlsx"

# Fonction Debug

In [4]:
def debug(message=" ") :
    ts = time.time()
    ts = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    print ("{} Debug -> ".format(ts)+str(message))

Fonction d'optention du code région via API
https://geo.api.gouv.fr/decoupage-administratif/regions#name

In [16]:
def get_code_region (param) :
    URL = 'https://geo.api.gouv.fr/communes?nom={ville}&fields=departement&boost=population&limit=5'.format(ville=param)
    reponse = requests.get (URL)
    return reponse

In [19]:
print (get_code_region('Marseille').text)
r = get_code_region('Marseille')

[{"nom":"Marseille","code":"13055","_score":4.833894444752829,"departement":{"code":"13","nom":"Bouches-du-Rhône"}},{"nom":"Marseillette","code":"11220","_score":0.6157962970305906,"departement":{"code":"11","nom":"Aude"}},{"nom":"Marseilles-lès-Aubigny","code":"18139","_score":0.4723387123353396,"departement":{"code":"18","nom":"Cher"}},{"nom":"Marseille-en-Beauvaisis","code":"60387","_score":0.3013473248598236,"departement":{"code":"60","nom":"Oise"}}]


In [33]:
r.json()

[{'nom': 'Marseille',
  'code': '13055',
  '_score': 4.833894444752829,
  'departement': {'code': '13', 'nom': 'Bouches-du-Rhône'}},
 {'nom': 'Marseillette',
  'code': '11220',
  '_score': 0.6157962970305906,
  'departement': {'code': '11', 'nom': 'Aude'}},
 {'nom': 'Marseilles-lès-Aubigny',
  'code': '18139',
  '_score': 0.4723387123353396,
  'departement': {'code': '18', 'nom': 'Cher'}},
 {'nom': 'Marseille-en-Beauvaisis',
  'code': '60387',
  '_score': 0.3013473248598236,
  'departement': {'code': '60', 'nom': 'Oise'}}]

# Fonction de chargement et d'agrégation des fichiers txt

In [5]:
def df_get (filepath) :
    
    files = [f for f in os.listdir(filepath_data) if f.endswith(".txt")]
    
    for i, file in enumerate(files):
        debug("Agregation dans le dataframe FONCIER du fichier :" + file)
        list_df.append(pd.read_csv(filepath_data + chr(92) + file, delimiter='|', low_memory=False,
                                   decimal=",",
                                   dtype={"Surface_Carrez_du_1er_lot": np.float64,
                                         "Surface_Carrez_du_2eme_lot": np.float64,
                                         "Surface_Carrez_du_3eme_lot": np.float64,
                                         "Surface_Carrez_du_4eme_lot": np.float64,
                                         "Surface_Carrez_du_5eme_lot": np.float64}
                                  ))

        debug("Nombre de ligne chargées :" + str(len(list_df[i])))
        debug()

    debug("Fusion des Dataframes")
    df = pd.concat(list_df)
    debug()

    debug("Netoyage des headers de colonne")
    df.rename(columns=lambda x: x.replace(' ', '_').replace('/', '_'), inplace=True)
    debug()
    return df

# Fonction d'addition des colonnes et suppression des colonnes additionées

In [6]:
def df_add_col (df,col_list,col_tot) :
    # Certaine colonnes, dont la suppression est demandée maintenant, ont potentiellement déjà été supprimées parce que vide
    # Oblige à faire un examen de chaque colonne , 1 par une , et non un drop de la liste complète.
    col_to_sum = [col for col in df.columns if col in col_list]
    debug ("Addition des colonnes {c} dans {t}".format(c=str(col_to_sum),t=col_tot))
    df[col_tot]=df[col_to_sum].sum(axis=1)
    # debug ("suppresion des colonnes "+str(col_to_sum))
    # df.drop(col_to_sum,axis=1,inplace=True)
    return df
    

# Fonction de suppression des colonnes vides

In [7]:
def df_del_empty_col (df) :
    to_remove = [col for col in df.columns if df[col].value_counts().empty]
    debug("Supression des colonnes vides :"+str(to_remove))
    df.drop (to_remove,axis=1,inplace=True)
    return df

# Fonction de génération d'un fichier sample

In [8]:
def df_gen_sample (df,nb_rows,filename,filepath) :
    
    debug ("Generation du fichier d'échantillon pour analyse sous excel :")
    debug (filepath+filename)
    # Init de l'engine
    ts = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H_%M_%S')
    writer_sample = pd.ExcelWriter(filepath+str(ts)+"_"+filename+".xlsx", engine='xlsxwriter')

    #génération 
    df.sample(n=nb_rows,random_state=1).to_excel(writer_sample,sheet_name="Sample")

    #instanciation de l'onglet "sample"
    worksheet_sample = writer_sample.sheets['Sample']

    #Auto filter des colones
    worksheet_sample.autofilter("A1:AS"+str(nb_rows-1))

    writer_sample.save()
    writer_sample.close()

# Generation du fichier d'analyse

In [9]:
def df_gen_analyse (df,filepath,filename) :
    debug("Generation du fichier d'analyse")
    path = filepath + filename
    writer_analyse = pd.ExcelWriter(path, engine='xlsxwriter')
    debug("Creation de l'onglet INFO ")
    workbook_foncier  = writer_analyse.book
    worksheet_info=workbook_foncier.add_worksheet("INFO")
    worksheet_drop=workbook_foncier.add_worksheet("DROP")
    worksheet_describep=workbook_foncier.add_worksheet("DESCRIBE")
    
    debug("Alimentation de l'onglet INFO ")
    cell_format = workbook_foncier.add_format()
    cell_format.set_align('vjustify')
    buf = io.StringIO()
    df.info(buf=buf)
    s = buf.getvalue()
    worksheet_info.write(0,0,s,cell_format)
    
    debug("Generation des counts par colonne")
    for i,col in enumerate(df.columns):
        debug ("Ajout de la colonne+count au fichier Excel : "+col)
        df[col].value_counts().to_excel(writer_analyse,sheet_name=col)
            
    writer_analyse.save()
    writer_analyse.close()
    debug("Sauvegarde de l'analyse sous excel")


# Main

In [11]:
debug ("Start")
debug ("************** Chargement, agrégation, et suppression des colonnes vides")
debug ()
df_foncier = df_get (filepath_data)
debug ("End")

2021-05-11 22:22:46 Debug -> Start
2021-05-11 22:22:46 Debug -> ************** Chargement, agrégation, et suppression des colonnes vides
2021-05-11 22:22:46 Debug ->  
2021-05-11 22:22:46 Debug -> Agregation dans le dataframe FONCIER du fichier :valeursfoncieres-2016.txt
2021-05-11 22:23:00 Debug -> Nombre de ligne chargées :2939478
2021-05-11 22:23:00 Debug ->  
2021-05-11 22:23:00 Debug -> Agregation dans le dataframe FONCIER du fichier :valeursfoncieres-2017.txt
2021-05-11 22:23:15 Debug -> Nombre de ligne chargées :3382812
2021-05-11 22:23:15 Debug ->  
2021-05-11 22:23:15 Debug -> Agregation dans le dataframe FONCIER du fichier :valeursfoncieres-2018.txt
2021-05-11 22:23:29 Debug -> Nombre de ligne chargées :3329147
2021-05-11 22:23:29 Debug ->  
2021-05-11 22:23:29 Debug -> Agregation dans le dataframe FONCIER du fichier :valeursfoncieres-2019.txt
2021-05-11 22:23:43 Debug -> Nombre de ligne chargées :3533211
2021-05-11 22:23:43 Debug ->  
2021-05-11 22:23:43 Debug -> Agregation 

debug ("************** Addition des surfaces Carrez")

df_foncier=df_add_col (df_foncier,["Surface_Carrez_du_1er_lot",
                                    "Surface_Carrez_du_2eme_lot",
                                    "Surface_Carrez_du_3eme_lot",
                                    "Surface_Carrez_du_4eme_lot",
                                    "Surface_Carrez_du_5eme_lot"],"Total_surface_carrez")

debug("end")

debug ("************** Suppression des colonnes inutiles")

to_remove =['No_disposition','No_voie','B_T_Q','Type_de_voie','Voie','Code_postal','Commune','Code_departement','Prefixe_de_section',
             'Section','No_plan','No_Volume','1er_lot','2eme_lot','3eme_lot','4eme_lot','5eme_lot','Code_type_local']

for col in to_remove :
    try :
        df_foncier.drop(col,axis=1,inplace=True)
    except :
        debug ("Colonne {c} non présente dans le data frame".format(c=col))

In [14]:
debug ("************** Génération du fichier de Sample")
ts = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H_%M_%S')
df_gen_sample (df_foncier,500000,str(ts)+"_"+"Sample Foncier",filepath_processed)
debug ("end")
debug ()

2021-05-11 22:26:18 Debug -> ************** Génération du fichier de Sample
2021-05-11 22:26:18 Debug -> Generation du fichier d'échantillon pour analyse sous excel :
2021-05-11 22:26:18 Debug -> C:\Users\gille\JupyterProjects\Foncier\data\processed\2021-05-11_22_26_18_Sample Foncier
2021-05-11 22:30:59 Debug -> end
2021-05-11 22:30:59 Debug ->  


C:\Users\gille\anaconda3\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


# Sauvegarde du DF au format CSV

df_foncier.to_csv (filepath_interim+'df_foncier.txt')